In [1]:
import shutil, glob, re, os
import pandas as pd

<h2> Code to place n-way matching files correctly in data directory </h2>

- n-way-matching directory should be in the current directory.  
- Provide target data directory

In [2]:
# Provide target data directory (including "prod")
data_direc = os.path.join("..", "..", "data", "OSCA", "prod")

In [5]:
def get_target_direcs(matching_files, data_direc):
    """
    Returns the target directories (mouse/session specific) for each json
    """
    
    # get file names
    matching_files_local = [os.path.split(matching_file)[-1] for matching_file in matching_files]
    
    # identify target directories
    file_name_tokens = [matching_file_local.split("_") for matching_file_local in matching_files_local]
    mouse_ids = [tokens[tokens.index("mouse") + 1] for tokens in file_name_tokens]
    sess_ids = [tokens[tokens.index("session") + 1] for tokens in file_name_tokens]
    target_direcs = [glob.glob(
        os.path.join(
            data_direc, 
            f"mouse_{mouse_ids[i]}", 
            f"ophys_session_{sess_ids[i]}", 
            f"ophys_experiment_*", 
            "processed")
            )[0] for i in range(len(file_name_tokens))]
    
    
    # identify target paths
    matching_file_targets = [
        os.path.join(target_direc, matching_file_local) 
        for (target_direc, matching_file_local) in zip(target_direcs, matching_files_local)
    ]
    
    return matching_file_targets

In [6]:
matching_files_direc = "nway-matched-dfs--iou-min-0.3"
matching_files = glob.glob(os.path.join(matching_files_direc, "*.json"))
matching_file_targets = get_target_direcs(matching_files, data_direc)

# copy files over
for (matching_file, matching_file_target) in zip(matching_files, matching_file_targets):
    shutil.copy(matching_file, matching_file_target)